In [1]:
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(
    host = "localhost",
    database = "baseball",
    user = "karisch",
    password = "cocacola",
    port = 5432
)

In [11]:
with conn.cursor() as cur:
    cur.execute("""
        SELECT pitches."typeCode"
        FROM games INNER JOIN pitches
            ON pitches."gamePk" = games.pk
        WHERE games."season" >= 2019
        ;
    """)
    raw = cur.fetchall()
    #cols = [desc[0] for desc in cur.description]
    #full = pd.DataFrame(raw, columns=cols)

In [22]:
typeDict = {}
for pitch in types:
    if pitch not in typeDict.keys():
        typeDict[pitch] = 1
    else:
        typeDict[pitch] += 1

In [25]:
typePerc = {}
for pitch, num in typeDict.items():
    typePerc[pitch] = num / len(types) * 100
typePerc

{'FF': 35.72434586555009,
 'FT': 6.086037571874268,
 'SL': 17.779459158644638,
 'KC': 2.281489838974959,
 'CU': 8.961389438937605,
 'CH': 11.183416841210517,
 'SI': 10.014274796503152,
 'FC': 6.412795096635988,
 'FS': 1.5068581916177022,
 'KN': 0.02125019773166051,
 'EP': 0.005431664891052237,
 'CS': 0.01943964276797643,
 'FO': 0.0037164022938778467,
 'SV': 9.529236650968837e-05}

In [17]:
types = [x[0] for x in raw if x[0] is not None]

In [ ]:
with conn.cursor() as cur:
    cur.execute("""
        SELECT
            pitches.*
        FROM atbats INNER JOIN pitches
            ON atbats."gamePk" = pitches."gamePk" AND atbats."atBatIndex" = pitches."atBatIndex"
        WHERE atbats."batterID" = 514888
        ;
    """)
    data = cur.fetchall()
    cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(data, columns=cols)
df = df[~df['isBall'].isnull()] #filter out all non-pitches (i.e. pickoff attempts)
df.sort_values(by=['gamePk', 'atBatIndex', 'pitchIndex'], ascending=True, inplace=True) #put in order (will need to revisit for more logical order (by date))
df.reset_index(drop=True, inplace=True)

df['newGame'] = df['gamePk'].reset_index(drop=True) != pd.Series(-100).append(df['gamePk'][:-1]).reset_index(drop=True) #new variable bool that says whether the game is different from prior entry
atBatChange = df['atBatIndex'].reset_index(drop=True) != pd.Series(-100).append(df['atBatIndex'][:-1]).reset_index(drop=True)
df['newAtBat'] = df['newGame'] | atBatChange #new variable for different atbat vs prior
df['priorTypeCode'] = pd.Series('na').append(df['typeCode'][:-1]).reset_index(drop=True)
df.loc[df['newAtBat'], 'priorTypeCode'] = 'na'

In [ ]:
result = df[['typeCode', 'priorTypeCode', 'callCode', 'gamePk']].groupby(by=['typeCode', 'priorTypeCode', 'callCode'], as_index = False).count()
result.columns = list(result.columns)[:-1] + ['count']
result.to_csv('penis.csv')

In [ ]:
[56, 7] + [5]

In [ ]:
result

In [ ]:
df.to_csv('penis.csv')

In [ ]:
df['callDesc'].unique()